# Importação das Bibliotecas

In [1]:
import pandas as pd
import numpy as np

Para o modelo de machine learning será utilizado o arquivo com os dados já tratados.

In [2]:
df=pd.read_csv('E:\\Usuario\\Desktop\\DesafioDS\\dados_menos_outliers.csv')
df.head(5)

,id,nome,host_id,host_name,bairro_group,bairro,latitude,longitude,room_type,price,minimo_noites,numero_de_reviews,ultima_review,reviews_por_mes,calculado_host_listings_count,disponibilidade_365
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
1,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,Não Informado,0.00,1,365
2,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
3,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,2019-06-22,0.59,1,129
4,5203,Cozy Clean Guest Room - Family Apt,7490,MaryEllen,Manhattan,Upper West Side,40.80178,-73.96723,Private room,79,2,118,2017-07-21,0.99,1,0


In [3]:
#Importação do pycaret 
import pycaret
from pycaret.regression import *

Desenvolvimento do modelo com a seleção das colunas a serem mantidas e justificativa para a inclusão de cada uma delas:

**bairro_group:** Durante a análise exploratória, observou-se que regiões próximas a pontos turísticos tendem a ter valores mais elevados;

**bairro:** Bairros localizados mais próximos de pontos turísticos tendem a ter valores mais elevados, especialmente entre o Central Park e a Estátua da Liberdade.

**room_type:** Demonstrou que entire room/apto tem uma média de valor maior que as outras categorias.

**price:** target

**minimo_noites:** Foi identificado uma correlação próxima de zero, porém positiva. Diante disso, optou-se por mantê-los e analisá-los no modelo.

**disponibilidade_365:** Foi identificada uma correlação próxima de zero, porém positiva, e por isso optou-se por mantê-la e analisá-la no modelo, especialmente porque, ao agrupar os valores e plotar no gráfico de barras, observou-se um padrão significativo.

In [4]:
modelo = df[['minimo_noites',  'disponibilidade_365','price', 'bairro_group', 'room_type']]

### Preparando os dados para modelagem

Na preparação do modelo, foram configurados os seguintes parâmetros:

- O banco de dados foi definido como modelo usando o parâmetro *data*.
- A variável alvo foi definida como 'price' usando o parâmetro *target*.
- Para normalizar os dados e mantê-los na mesma escala, foi utilizado o método 'zscore' com o parâmetro *normalize*.
- Os dados foram transformados utilizando o método 'yeo-johnson' através do parâmetro *transformation*.
- Para garantir a reprodutibilidade dos resultados, foi atribuído um identificador de sessão único (980) usando o parâmetro *session_id*.

In [5]:
prep_model = setup(data=modelo, target= 'price',  normalize= True, normalize_method='zscore',
                   transformation=True, session_id=980)

,Description,Value
0,Session id,980
1,Target,price
2,Target type,Regression
3,Original data shape,"(44498, 5)"
4,Transformed data shape,"(44498, 11)"
5,Transformed train set shape,"(31148, 11)"
6,Transformed test set shape,"(13350, 11)"
7,Numeric features,2
8,Categorical features,2
9,Preprocess,True


Utilizou-se a transformação de Yeo-Johnson pois ela ajuda a ajustar os dados para torná-los mais simétricos e próximos de uma distribuição normal e com isso melhorar o desempenho dos modelos. Como a base de dados, mesmo tratada, possui vários outliers o uso desse método reduz o efeito deles tornando mais adequado para a modelagem. 

Comparar vários modelos de machine learning e selecionar o melhor deles.

In [6]:
melhores_modelo = compare_models(n_select=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,46.4430,4845.4811,69.5943,0.3974,0.4413,0.4051,2.3100
lightgbm,Light Gradient Boosting Machine,46.4547,4863.0095,69.7223,0.3951,0.4417,0.4044,1.3970
lar,Least Angle Regression,47.8400,5048.0776,71.0361,0.3723,0.4588,0.4212,1.1650
br,Bayesian Ridge,47.8388,5048.0766,71.0361,0.3723,0.4587,0.4212,0.9890
ridge,Ridge Regression,47.8400,5048.0775,71.0361,0.3723,0.4588,0.4212,1.2610
lr,Linear Regression,47.8472,5049.5876,71.0468,0.3721,0.4588,0.4211,2.0050
lasso,Lasso Regression,47.7629,5056.9292,71.0980,0.3712,0.4563,0.4234,1.0950
llar,Lasso Least Angle Regression,47.7629,5056.9297,71.0980,0.3712,0.4563,0.4234,0.4150
en,Elastic Net,49.0082,5220.9384,72.2404,0.3509,0.4754,0.4612,0.6820
huber,Huber Regressor,45.6616,5312.1082,72.8639,0.3397,0.4428,0.3603,0.5590


Realizando a comparação dos três melhores modelos com a amostra.

In [7]:
amostra = pd.DataFrame({'id': [2595],
                        'nome': ['Skylit Midtown Castle'],
                        'host_id': [2845],
                        'host_name': ['Jennifer'],
                        'bairro_group': ['Manhattan'],
                        'bairro': ['Midtown'], 
                        'latitude': [40.75362], 
                        'longitude': [-73.98377],
                        'room_type': ['Entire home/apt'], 
                        'price': [225], 
                        'minimo_noites': [1],
                        'numero_de_reviews': [45], 
                        'ultima_review': ['2019-05-21'],
                        'reviews_por_mes': [0.38], 
                        'calculado_host_listings_count': [2],
                        'disponibilidade_365': [355]}
                              )

In [8]:
previsoes_melhores_modelos = [predict_model(modelo, data=amostra) for modelo in melhores_modelo]

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,29.6304,877.9579,29.6304,nan,0.1232,0.1317


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,5.1041,26.0518,5.1041,nan,0.0223,0.0227


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Least Angle Regression,0.2287,0.0523,0.2287,nan,0.0010,0.0010


De acordo com o Erro Quadrático Médio(MSE) o que apresentou um menor valor foi o Least Angle Regression(LARS). Verificaremos ao comparar os valores.

In [9]:
for i, previsao in enumerate(previsoes_melhores_modelos):
    print(f"Resultados do Modelo {i+1}:")
    print(previsao)
    print("\n")


Resultados do Modelo 1:
     id                   nome  host_id host_name bairro_group   bairro  \
0  2595  Skylit Midtown Castle     2845  Jennifer    Manhattan  Midtown   

   latitude  longitude        room_type  minimo_noites  numero_de_reviews  \
0  40.75362 -73.983772  Entire home/apt              1                 45   

  ultima_review  reviews_por_mes  calculado_host_listings_count  \
0    2019-05-21             0.38                              2   

   disponibilidade_365  price  prediction_label  
0                  355    225        254.630355  


Resultados do Modelo 2:
     id                   nome  host_id host_name bairro_group   bairro  \
0  2595  Skylit Midtown Castle     2845  Jennifer    Manhattan  Midtown   

   latitude  longitude        room_type  minimo_noites  numero_de_reviews  \
0  40.75362 -73.983772  Entire home/apt              1                 45   

  ultima_review  reviews_por_mes  calculado_host_listings_count  \
0    2019-05-21             0.38    

Para melhor visualização inseriu os dados em um DataFrame

In [10]:
resultados_concatenados = pd.concat(previsoes_melhores_modelos, ignore_index=True)
resultados_concatenados[['price', 'prediction_label']]

,price,prediction_label
0,225,254.630355
1,225,230.104097
2,225,225.228671


In [11]:
#Calcular diferença
diferenca = (resultados_concatenados['price']-resultados_concatenados['prediction_label'])

#Calcular o erro percentual
erro_perc = (diferenca/resultados_concatenados['price'])*100

#Adicionar coluna no DataFrame
resultados_concatenados['erro_percentual'] = erro_perc

#Apresentar o DataFrame com o erro percentual
resultados_concatenados[['price', 'prediction_label', 'erro_percentual']]

,price,prediction_label,erro_percentual
0,225,254.630355,-13.169047
1,225,230.104097,-2.268488
2,225,225.228671,-0.101631


Apesar do 'Gradient Boosting Regressor' ter apresentado um melhor desempenho pelo pycaret. O 'Least Angle Regression' foi o que apresentou um menor erro em comparação a nossa amostra, sendo assim foi considerado o melhor para esse caso.

Um dos pró do LARS é ele ser eficaz em conjuntos de dados com um grande número de recursos em comparação com o número de amostras.Entretanto pode ser afetado por multicolinearidade e também pode não captar correlações não lineares 

Dos modelos utilizados no projeto o LightGBM é o único que pode ser utilizado como regressão e classificação, entretanto por se tratar de uma previsão de preço todas os modelos utilizam o algaritmo de regressão.

### Salvando como .pkl

Salvou apenas o modelo 3, pois foi o que mais se aproximou do valor real

In [33]:
import pickle

In [34]:
terceiro_modelo = melhores_modelo[2]

# Salvar o terceiro modelo em um arquivo .pkl

with open('modelo_machine_learning.pkl', 'wb') as file:
    pickle.dump(terceiro_modelo, file)
